In [1]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import time
import math
import community as community_louvain
import networkx.algorithms.community as modularity
from networkx.algorithms.community.quality import modularity
from collections import defaultdict


G = nx.read_edgelist("C:/Users/gnssl/Downloads/facebook_combined.txt", 
                     create_using=nx.Graph(), nodetype=int)

partition_louvain = community_louvain.best_partition(G)
len_partition = len(partition_louvain)

population_size = 200
generations = 500
crossover_prob = 1.0
mutation_prob = 0.05
local_optimization_frequency=5

min_partition_size = 5
max_partition_size = 14

def initialize_population():
    population = []
    for idx in range(population_size):
        if idx%4==0:
            partition_louvain = community_louvain.best_partition(G)
            partition = [partition_louvain[node] for node in G.nodes]
            population.append(partition)
        else:
            partition = [random.randint(0, max_partition_size) for _ in range(len(G.nodes))]
            population.append(partition)
    return population

def compute_fitness(partition):
    communities = {key: value for key, value in zip(partition_louvain.keys(), partition)}
    new_dict = defaultdict(list)

    for key, value in communities.items():
        new_dict[value].append(key)

    result_dict = dict(new_dict)
    fitness = nx.community.modularity(G, result_dict.values())
    return fitness


def select_parents(population, fitness_values):
    parent1, parent2 = random.sample(population, 2)
    if compute_fitness(parent1) > compute_fitness(parent2):
        return parent1
    else:
        return parent2

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(individual, mutation_prob, max_partition_size):
    for i in range(len(individual)):
        if random.random() < mutation_prob:
            # Mutate the community assignment of the node to a random community
            individual[i] = random.randint(0, max_partition_size - 1)
    return individual

def partition_to_communities(partition):
    communities = {}
    for node, community_id in enumerate(partition):
        if community_id in communities:
            communities[community_id].add(node)
        else:
            communities[community_id] = {node}
    return list(communities.values())



def genetic_algorithm():
    population = initialize_population()
    max_fitness_values = []  # Store max fitness values for each generation

    for generation in range(generations):
        fitness_values = [compute_fitness(partition) for partition in population]
        max_fitness = max(fitness_values)
        max_fitness_values.append(max_fitness)
        avg_fitness = sum(fitness_values) / len(fitness_values)
        print('Generation', generation + 1, 'Max:', max_fitness)
        print('Generation', generation + 1, 'Avg:', avg_fitness)

        new_population = []
        for _ in range(population_size // 2):
            parent1 = select_parents(population, fitness_values)
            parent2 = select_parents(population, fitness_values)
            if random.random() < crossover_prob:
                child1, child2 = crossover(parent1, parent2)
                
                child1 = mutate(child1, mutation_prob, max_partition_size)
                child2 = mutate(child2, mutation_prob, max_partition_size)
                
                min_fitness_index = min(range(len(fitness_values)), key=fitness_values.__getitem__)
                population[min_fitness_index] = child1
                second_min_fitness_index = min(range(len(fitness_values)), key=lambda x: fitness_values[x] if x != min_fitness_index else float('inf'))
                population[second_min_fitness_index] = child2
                
        
        if generation % local_optimization_frequency == 0:
            random_idx = random.randint(1,population_size//2)
            for idx in range(0, random_idx, population_size//random_idx):
                partition_louvain = community_louvain.best_partition(G)
                partition = [partition_louvain[node] for node in G.nodes]
                population[idx] = partition
       
    # Select the best partition from the final population
    best_partition = max(population, key=compute_fitness)
    return best_partition, max_fitness_values

start = time.time()
best_partition, max_fitness_values = genetic_algorithm()

best_communities = partition_to_communities(best_partition)
end = time.time()
print('time:', end - start)

print("Best Partition:", best_communities)

Generation 1 Max: 0.8349943827336102
Generation 1 Avg: 0.20836360978297783
Generation 2 Max: 0.8357836671335606
Generation 2 Avg: 0.2083855329108804
Generation 3 Max: 0.8357836671335606
Generation 3 Avg: 0.21155296704461135
Generation 4 Max: 0.8357836671335606
Generation 4 Avg: 0.21811408400337298
Generation 5 Max: 0.8357836671335606
Generation 5 Avg: 0.22128333628237143
Generation 6 Max: 0.8357836671335606
Generation 6 Avg: 0.22129208845390216
Generation 7 Max: 0.8357836671335606
Generation 7 Avg: 0.22491193626310146
Generation 8 Max: 0.8357836671335606
Generation 8 Avg: 0.22886680623915334
Generation 9 Max: 0.8357836671335606
Generation 9 Avg: 0.22887771576905497
Generation 10 Max: 0.8357836671335606
Generation 10 Avg: 0.23207032967679728
Generation 11 Max: 0.8357836671335606
Generation 11 Avg: 0.23548841434208664
Generation 12 Max: 0.8357836671335606
Generation 12 Avg: 0.2471583477172808
Generation 13 Max: 0.8357836671335606
Generation 13 Avg: 0.25253083947551563
Generation 14 Max: 

Generation 109 Max: 0.8357230295584268
Generation 109 Avg: 0.7580657082750493
Generation 110 Max: 0.8357230295584268
Generation 110 Avg: 0.7580989659329854
Generation 111 Max: 0.8357230295584268
Generation 111 Avg: 0.7592430351618369
Generation 112 Max: 0.8358355259798533
Generation 112 Avg: 0.7592701997545522
Generation 113 Max: 0.8358355259798533
Generation 113 Avg: 0.7600681331731276
Generation 114 Max: 0.8358355259798533
Generation 114 Avg: 0.7606448275948678
Generation 115 Max: 0.8358355259798533
Generation 115 Avg: 0.7605917912368393
Generation 116 Max: 0.8358355259798533
Generation 116 Avg: 0.7617693741331165
Generation 117 Max: 0.8358355259798533
Generation 117 Avg: 0.7621281815188048
Generation 118 Max: 0.8358355259798533
Generation 118 Avg: 0.7631505170558965
Generation 119 Max: 0.8358355259798533
Generation 119 Avg: 0.7637680913590988
Generation 120 Max: 0.8358355259798533
Generation 120 Avg: 0.764025234320958
Generation 121 Max: 0.8358355259798533
Generation 121 Avg: 0.7648

Generation 215 Max: 0.8358099104520256
Generation 215 Avg: 0.7842447921808481
Generation 216 Max: 0.8358099104520256
Generation 216 Avg: 0.7837450624545756
Generation 217 Max: 0.8349793603940692
Generation 217 Avg: 0.7839346256473375
Generation 218 Max: 0.8349793603940692
Generation 218 Avg: 0.784700471129649
Generation 219 Max: 0.8349793603940692
Generation 219 Avg: 0.7838025837389622
Generation 220 Max: 0.8349793603940692
Generation 220 Avg: 0.7848033425071494
Generation 221 Max: 0.8349793603940692
Generation 221 Avg: 0.7846584323502028
Generation 222 Max: 0.8349793603940692
Generation 222 Avg: 0.7852825552939282
Generation 223 Max: 0.8349793603940692
Generation 223 Avg: 0.7846533633274357
Generation 224 Max: 0.8349793603940692
Generation 224 Avg: 0.7844915439883654
Generation 225 Max: 0.8349793603940692
Generation 225 Avg: 0.7848983753418112
Generation 226 Max: 0.8349793603940692
Generation 226 Avg: 0.7854774834390451
Generation 227 Max: 0.8349793603940692
Generation 227 Avg: 0.7857

Generation 321 Max: 0.8355887404206412
Generation 321 Avg: 0.7890438403405727
Generation 322 Max: 0.8358638733442589
Generation 322 Avg: 0.7890601366850365
Generation 323 Max: 0.8358638733442589
Generation 323 Avg: 0.7896862221899599
Generation 324 Max: 0.8358638733442589
Generation 324 Avg: 0.7897815472491212
Generation 325 Max: 0.8358638733442589
Generation 325 Avg: 0.7898584461367164
Generation 326 Max: 0.8358638733442589
Generation 326 Avg: 0.7896120889609626
Generation 327 Max: 0.8358638733442589
Generation 327 Avg: 0.7898553945090473
Generation 328 Max: 0.8358638733442589
Generation 328 Avg: 0.7893188129055889
Generation 329 Max: 0.8358638733442589
Generation 329 Avg: 0.7888823973685726
Generation 330 Max: 0.8358638733442589
Generation 330 Avg: 0.7893049198085512
Generation 331 Max: 0.8358638733442589
Generation 331 Avg: 0.7901116227998577
Generation 332 Max: 0.8358638733442589
Generation 332 Avg: 0.7900099126841096
Generation 333 Max: 0.8358638733442589
Generation 333 Avg: 0.789

Generation 427 Max: 0.8357656052597495
Generation 427 Avg: 0.7907755877027987
Generation 428 Max: 0.8357656052597495
Generation 428 Avg: 0.7908712793653601
Generation 429 Max: 0.8357656052597495
Generation 429 Avg: 0.7905677774026646
Generation 430 Max: 0.8357656052597495
Generation 430 Avg: 0.7903123743354218
Generation 431 Max: 0.8357656052597495
Generation 431 Avg: 0.7903407233166689
Generation 432 Max: 0.8357656052597495
Generation 432 Avg: 0.7907850564374528
Generation 433 Max: 0.8357656052597495
Generation 433 Avg: 0.7906061513808896
Generation 434 Max: 0.8357656052597495
Generation 434 Avg: 0.7911160216079832
Generation 435 Max: 0.8357656052597495
Generation 435 Avg: 0.790357833949904
Generation 436 Max: 0.8357656052597495
Generation 436 Avg: 0.7906999015114732
Generation 437 Max: 0.8357656052597495
Generation 437 Avg: 0.79081308413215
Generation 438 Max: 0.8357656052597495
Generation 438 Avg: 0.7910720591739812
Generation 439 Max: 0.8357656052597495
Generation 439 Avg: 0.791171